In [ ]:
from ortools.algorithms.python import knapsack_solver
from random import randint
import numpy as np


def solve_problem(values, weights, capacities, method='ortools'):
    if not values or not weights:
        return None

    if method == 'dp':
        table = np.zeros(shape=(len(values) + 1, capacities + 1))
        current_items = [[[] for j in range(0, capacities + 1)] for k in range(0, len(values) + 1)]

        for i in range(0, len(values) + 1):
            for j in range(0, capacities + 1):
                if i == 0 or j == 0:
                    table[i][j] = 0
                elif weights[i - 1] <= j:

                    check1 = values[i - 1] + table[i - 1][j - weights[i - 1]]
                    check2 = table[i - 1][j]

                    table[i][j] = max(values[i - 1] + table[i - 1][j - weights[i - 1]], table[i - 1][j])
                    if values[i - 1] + table[i - 1][j - weights[i - 1]] > table[i - 1][j]:
                        current_items[i][j] = current_items[i - 1][j - weights[i - 1]].copy()
                        current_items[i][j].append(values[i - 1])
                    else:
                        current_items[i][j] = current_items[i - 1][j].copy()
                else:
                    table[i][j] = table[i - 1][j]
                    current_items[i][j] = current_items[i-1][j].copy()

        total_value = table[len(values)][capacities]
        print(current_items[len(values)][capacities])
        print(f'Total value: {total_value}')
        return total_value

    elif method == 'greedy':
        specific_value = sorted([(v/(w + 0.01), v, w, i) for v, w, i in zip(values, weights, range(0, len(values))) if w <= capacities], reverse=True, key=lambda x: x[0])

        packed_items = []
        packed_weights = []
        packed_indexes = []
        total_weight = 0

        while total_weight + specific_value[0][2] < capacities:
            packed_items.append(specific_value[0][1])
            packed_weights.append(specific_value[0][2])
            packed_indexes.append(specific_value[0][3])
            total_weight += specific_value[0][2]
            specific_value.pop(0)
            if not specific_value:
                break
        total_value = sum(packed_items)

        print(f'Total weight: {total_weight}')
        print(f'Packed items: {packed_items}')
        print(f'Packed_weights: {packed_weights}')
        print(f'Packed_indexes: {packed_indexes}')
        print('----------------------------')
        return packed_items, packed_weights, total_weight

    else:
        solver = knapsack_solver.KnapsackSolver(
            knapsack_solver.SolverType.KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
            "KnapsackExample",
        )

        weights = [weights]
        capacities = [capacities]

        solver.init(values, weights, capacities)
        computed_value = solver.solve()

        packed_items = []
        packed_indexes = []
        packed_weights = []
        total_weight = 0

        print(f'Total value = {computed_value}')

        for i in range(len(values)):
            if solver.best_solution_contains(i):
                packed_items.append(values[i])
                packed_indexes.append(i)
                packed_weights.append(weights[0][i])
                total_weight += weights[0][i]
        print(f'Total weight: {total_weight}')
        print(f'Packed items: {packed_items}')
        print(f'Packed_weights: {packed_weights}')
        print(f'Packed_indexes: {packed_indexes}')
        print('----------------------------')
        return packed_items, packed_weights, packed_indexes, total_weight


def make_affinity_constraints(values, weights, num_of_constraints):

    if len(values) < num_of_constraints/4:
        return None

    new_values = values.copy()
    new_weights = weights.copy()
    identifier = {}
    all_added = set()
    for i in range(0, num_of_constraints):
        num_of_items = randint(2, 4)
        pair = set()
        j = 0
        while j < num_of_items:
            item_num = randint(0, len(values)-1)
            if item_num not in pair and item_num not in all_added:
                pair.add(item_num)
                all_added.add(item_num)
                j += 1
        summ_v = 0
        summ_w = 0
        vals = []
        for item in pair:
            vals.append(values[item])
            summ_v += values[item]
            summ_w += weights[item]
        identifier[(summ_v, summ_w)] = vals

    arr = sorted(list(all_added), reverse=True)
    for i in arr:
        del new_values[i]
        del new_weights[i]

    for key in identifier.keys():
        new_values.append(key[0])
        new_weights.append(key[1])

    return new_values, new_weights, identifier


def main():
    #name = 'data/knapPI_1_100_1000_1.txt'
    name = 'data/data.txt'
    num_of_items = 0
    capacity = 0
    opt_solution = 0
    values = []
    weights = []

    with open(name) as f:
        num_of_items = int(f.readline())
        capacity = int(f.readline())
        opt_solution = int(f.readline())
        for i in range(0, num_of_items):
            value, weight = map(int, f.readline().split(' '))
            values.append(value)
            weights.append(weight)

    new_values, new_weights, identifier = make_affinity_constraints(values, weights, 2)
    print(new_values)
    print(new_weights)
    print(identifier)

    print(f'Known solution: {opt_solution}')

    print(f'Solver solution without constraints by ortools: ')
    solve_problem(values, weights, capacity, 'ortools')
    print(f'Solver solution with constraints by ortools: ')
    solve_problem(new_values, new_weights, capacity, 'ortools')

    print(f'Solver solution without constraints by greedy algorithm: ')
    solve_problem(values, weights, capacity, 'greedy')
    print(f'Solver solution with constraints by greedy algorithm: ')
    solve_problem(new_values, new_weights, capacity, 'greedy')

    print(f'Solver solution without constraints by dp: ')
    solve_problem(values, weights, capacity, 'dp')
    print(f'Solver solution with constraints by dp: ')
    solve_problem(new_values, new_weights, capacity, 'dp')


if __name__ == "__main__":
    main()
